In [1]:
import win32com.client
import pandas as pd
from datetime import datetime
from datetime import timedelta
instCpCybos = win32com.client.Dispatch('CpUtil.CpCybos')
instCpStockCode = win32com.client.Dispatch('CpUtil.CpStockCode')
instCpCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
instStockChart = win32com.client.Dispatch('CpSysDib.StockChart')


In [3]:
# 크레온 API 연결 확인
# <리턴값 종류 확인 필요>
def check_connection():
    if instCpCybos.IsConnect == 1:
        print('정상적으로 연결')
    else:
        print('연결 실패')
        
check_connection()

정상적으로 연결


In [4]:
# 종목 이름으로 찾기
def search_by_name(stock_name):
    count = instCpStockCode.GetCount()
    for i in range(count):
        if instCpStockCode.GetData(1,i) == stock_name:
            # GetData(x,y)
            # x : 0 => 종목 코드
            # x : 1 => 종목명
            # x : 2 => ISIN 코드(ISIN : 국제 증권 식별 번호)
            print(instCpStockCode.GetData(0,i))
            print(instCpStockCode.GetData(1,i))

search_by_name('삼성전자')

A005930
삼성전자


In [5]:
# 특정 코드의 종목 찾기
def search_by_code(stock_code):
    count = instCpStockCode.GetCount()
    for i in range(count):
        if instCpStockCode.GetData(0,i) == stock_code:
            # GetData(x,y)
            # x : 0 => 종목 코드
            # x : 1 => 종목명
            # x : 2 => ISIN 코드(ISIN : 국제 증권 식별 번호)
            print(instCpStockCode.GetData(0,i))
            print(instCpStockCode.GetData(1,i))

search_by_code('A005930')

A005930
삼성전자


In [6]:
# 종목명 전체 리스트 가져오기
# <선물이랑 ETF 쪽은 구분 필요>
def get_stock_list():
    count = instCpStockCode.GetCount()
    stock_list = []
    
    for i in range(count):
        stock_list.append(instCpStockCode.GetData(1,i))
    
    return stock_list

get_stock_list()

['동화약품',
 'KR모터스',
 '경방',
 '메리츠화재',
 '삼양홀딩스',
 '삼양홀딩스우',
 '하이트진로',
 '하이트진로2우B',
 '유한양행',
 '유한양행우',
 'CJ대한통운',
 '하이트진로홀딩스',
 '하이트진로홀딩스우',
 '두산',
 '두산우',
 '두산2우B',
 '성창기업지주',
 'DL',
 'DL우',
 '유유제약',
 '유유제약1우',
 '유유제약2우B',
 '일동홀딩스',
 '한국앤컴퍼니',
 '삼천당제약',
 '기아',
 '대유플러스',
 '노루홀딩스',
 '노루홀딩스우',
 '한화손해보험',
 '삼화페인트',
 '롯데손해보험',
 '대원강업',
 '중앙에너비스',
 '조선내화',
 '대동',
 '가온전선',
 '삼일제약',
 '흥국화재',
 '흥국화재우',
 '흥국화재2우B',
 'CS홀딩스',
 '동아쏘시오홀딩스',
 '천일고속',
 'SK하이닉스',
 '영풍',
 'LS네트웍스',
 '유수홀딩스',
 '현대건설',
 '현대건설우',
 '이화산업',
 '경남기업',
 '삼성화재',
 '삼성화재우',
 '화천기공',
 '강남제비스코',
 '한화',
 '한화우',
 '한화3우B',
 '보해양조',
 '유니온',
 '전방',
 '한국주철관',
 'DB하이텍',
 'DB하이텍1우',
 '신라섬유',
 '페이퍼코리아',
 'CJ',
 'CJ우',
 'CJ4우(전환)',
 'JW중외제약',
 'JW중외제약우',
 'JW중외제약2우B',
 '대한방직',
 '만호제강',
 'LX인터내셔널',
 '대한제분',
 '국보',
 '유진투자증권',
 '금호전기',
 '동국제강',
 'GS글로벌',
 '남광토건',
 '부국증권',
 '부국증권우',
 '상상인증권',
 '백광산업',
 '삼성제약',
 'SG글로벌',
 'KG케미칼',
 '태원물산',
 '세아베스틸지주',
 '대한전선',
 '현대해상',
 'BYC',
 'BYC우',
 '삼부토건',
 '천지산업',
 '현대차증권',
 'SK증권',
 'SK증권우',
 '동양',
 '동양우',
 '

In [7]:
codelist = instCpCodeMgr.GetStockListByMarket(1)
kospi = {}
for code in codelist:
    name = instCpCodeMgr.CodeToName(code)
    kospi[code] = name
    
kospi

{'A000020': '동화약품',
 'A000040': 'KR모터스',
 'A000050': '경방',
 'A000060': '메리츠화재',
 'A000070': '삼양홀딩스',
 'A000075': '삼양홀딩스우',
 'A000080': '하이트진로',
 'A000087': '하이트진로2우B',
 'A000100': '유한양행',
 'A000105': '유한양행우',
 'A000120': 'CJ대한통운',
 'A000140': '하이트진로홀딩스',
 'A000145': '하이트진로홀딩스우',
 'A000150': '두산',
 'A000155': '두산우',
 'A000157': '두산2우B',
 'A000180': '성창기업지주',
 'A000210': 'DL',
 'A000215': 'DL우',
 'A000220': '유유제약',
 'A000225': '유유제약1우',
 'A000227': '유유제약2우B',
 'A000230': '일동홀딩스',
 'A000240': '한국앤컴퍼니',
 'A000270': '기아',
 'A000300': '대유플러스',
 'A000320': '노루홀딩스',
 'A000325': '노루홀딩스우',
 'A000370': '한화손해보험',
 'A000390': '삼화페인트',
 'A000400': '롯데손해보험',
 'A000430': '대원강업',
 'A000480': '조선내화',
 'A000490': '대동',
 'A000500': '가온전선',
 'A000520': '삼일제약',
 'A000540': '흥국화재',
 'A000545': '흥국화재우',
 'A000547': '흥국화재2우B',
 'A000590': 'CS홀딩스',
 'A000640': '동아쏘시오홀딩스',
 'A000650': '천일고속',
 'A000660': 'SK하이닉스',
 'A000670': '영풍',
 'A000680': 'LS네트웍스',
 'A000700': '유수홀딩스',
 'A000720': '현대건설',
 'A000725': '현대건설우

In [8]:
for i, code in enumerate(codelist):
    secondCode = instCpCodeMgr.GetStockSectionKind(code)
    name = instCpCodeMgr.CodeToName(code)
    print(i, code, secondCode, name)

0 A000020 1 동화약품
1 A000040 1 KR모터스
2 A000050 1 경방
3 A000060 1 메리츠화재
4 A000070 1 삼양홀딩스
5 A000075 1 삼양홀딩스우
6 A000080 1 하이트진로
7 A000087 1 하이트진로2우B
8 A000100 1 유한양행
9 A000105 1 유한양행우
10 A000120 1 CJ대한통운
11 A000140 1 하이트진로홀딩스
12 A000145 1 하이트진로홀딩스우
13 A000150 1 두산
14 A000155 1 두산우
15 A000157 1 두산2우B
16 A000180 1 성창기업지주
17 A000210 1 DL
18 A000215 1 DL우
19 A000220 1 유유제약
20 A000225 1 유유제약1우
21 A000227 1 유유제약2우B
22 A000230 1 일동홀딩스
23 A000240 1 한국앤컴퍼니
24 A000270 1 기아
25 A000300 1 대유플러스
26 A000320 1 노루홀딩스
27 A000325 1 노루홀딩스우
28 A000370 1 한화손해보험
29 A000390 1 삼화페인트
30 A000400 1 롯데손해보험
31 A000430 1 대원강업
32 A000480 1 조선내화
33 A000490 1 대동
34 A000500 1 가온전선
35 A000520 1 삼일제약
36 A000540 1 흥국화재
37 A000545 1 흥국화재우
38 A000547 1 흥국화재2우B
39 A000590 1 CS홀딩스
40 A000640 1 동아쏘시오홀딩스
41 A000650 1 천일고속
42 A000660 1 SK하이닉스
43 A000670 1 영풍
44 A000680 1 LS네트웍스
45 A000700 1 유수홀딩스
46 A000720 1 현대건설
47 A000725 1 현대건설우
48 A000760 1 이화산업
49 A000810 1 삼성화재
50 A000815 1 삼성화재우
51 A000850 1 화천기공
52 A000860 1 강남제비스코
53 A000880

In [9]:
def get_stock_info(stock_code, start_day, end_day, type):
    
    instStockChart.SetInputValue(0, stock_code) # 종목명
    instStockChart.SetInputValue(1, ord('1')) # 1 : 기간으로 요청, 2: 개수로 요청
    instStockChart.SetInputValue(3, start_day) # 요청 시작일
    instStockChart.SetInputValue(2, end_day) # 요청 종료일
    # instStockChart.SetInputValue(4, request_num) # 요청 개수
    '''
    # 요청할 데이터 종류(리스트 형태로 요청 가능)
    0 : 날짜, 1 : 시간 - hhmm, 2 : 시가, 3 : 고가, 4 : 저가, 5 : 종가
    6 : 전일대비, 8 : 거래량, 9 : 거래대금, 13 : 시가총액
    '''
    instStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 6, 8, 9, 13])
    instStockChart.SetInputValue(6, ord(type)) # 'D' : 일봉, 'm' : 분봉
    instStockChart.SetInputValue(9, ord('1'))

    instStockChart.BlockRequest() # 위 정보로 요청

    numrow, numcolumn = instStockChart.GetHeaderValue(3), instStockChart.GetHeaderValue(2)

    index = []
    for i in range(numrow):
        index_ = str(instStockChart.GetDataValue(0,i))
        index.append(index_)

    stock_info = pd.DataFrame(columns=numcolumn[1:], index=index)

    for num in range(numrow):
        for col in range(len(numcolumn)):
            # 1,2,3,4,5,6,7,8,9, 10
            stock_info.iloc[num, col-1] = str(instStockChart.GetDataValue(col,num))
    
    return stock_info

In [10]:
donghwa = get_stock_info('A005930', '20221029', '20221031', 'm')
donghwa


,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,시가총액
20221031,1530,59400,59400,59400,59400,0,1670518,99228760000,0
20221031,1520,59400,59500,59400,59400,0,20508,1219300000,0
20221031,1519,59600,59600,59400,59400,0,33922,2018480000,0
20221031,1518,59500,59600,59500,59600,0,54361,3235660000,0
20221031,1517,59500,59600,59400,59600,0,92564,5507840000,0
...,...,...,...,...,...,...,...,...,...
20221031,905,58500,58800,58400,58700,0,339821,19919660000,0
20221031,904,58400,58500,58300,58400,0,167952,9810220000,0
20221031,903,58500,58600,58400,58400,0,168992,9885660000,0
20221031,902,58300,58500,58200,58400,0,239901,14004540000,0


In [11]:
def get_stock_info_auto(stock_code, start_day, end_day, type):
    
    print(datetime.strptime('20221031', '%YYYY%d%m'))
    
    end_day - start_day
    
    for day in 
        get_stock_info(stock_code, start_day, end_day, type)
    

SyntaxError: invalid syntax (348577546.py, line 7)

In [12]:
d1 = "20221031"
f1 = "%Y%m%d"

d2 = "20220630"
f2 = "%Y%m%d"

r1 = datetime.strptime(d1,f1)
r2 = datetime.strptime(d2,f2)

In [15]:
from datetime import timedelta


c1 = timedelta(days=5)


2022-10-26 00:00:00
